In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from time import sleep

from selenium.webdriver.common.proxy import Proxy, ProxyType

### Настройки webdriver

In [ ]:
""" Настройки прокси """
# при необходимости настраиваем прокси
prox = Proxy()
prox.proxy_type = ProxyType.MANUAL
prox.http_proxy = "ip_addr:port"
prox.socks_proxy = "ip_addr:port"
prox.ssl_proxy = "ip_addr:port"

capabilities = webdriver.DesiredCapabilities.CHROME
prox.add_to_capabilities(capabilities)

driver = webdriver.Chrome(desired_capabilities=capabilities)

In [14]:
""" Параметры браузера """
# Запускаем веб-драйвер (Chrome)

chrome_options = Options()

# Параметры для безоконного режима
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("--no-sandbox") # linux only
chrome_options.add_argument("--headless=new") # for Chrome >= 109
chrome_options.add_argument("--headless")
chrome_options.headless = True # also works

In [16]:
""" Параметры ожидания загрузки элемента """
# Ждем загрузки результатов поиска

# Ожидаем появления объекта карточек объектов
# results_loaded = WebDriverWait(driver, 20).until(
#     EC.presence_of_element_located((By.CLASS_NAME, "items-items-kAJAg"))
# )

# Ожидаем появления элементов пагинации
results_loaded = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div/div[4]/div/div[2]/div[3]/div[3]/div[4]'))
) 

### Запускаем парсер

In [20]:
""" Функция сбора данных со страницы"""

def get_data(elements):

    for num,el in enumerate(elements[::],1):
        
        id = el.get_attribute("id")
        title = el.find_element(By.XPATH,f'//*[@id="{id}"]/div/div/div[2]/div[2]/div/a/h3').text
        price = el.find_element(By.TAG_NAME,"p").text
        description = el.find_element(By.CSS_SELECTOR,"[class^='iva-item-descriptionStep-']").text
        geo = el.find_element(By.CSS_SELECTOR,"[class^='geo-root-']").text
        link = el.find_element(By.XPATH, f'//*[@id="{id}"]/div/div/div[2]/div[2]/div/a').get_attribute('href')
        
        print(num)
        print("Заголовок:", title)
        print("Описание:", description)
        print("Цена:", price)
        print("Ссылка:", link)
        print("Адрес:", geo)
        print("\n")


In [21]:

""" Параметры поиска """

region = "all" # регион поиска
search_query = "дом" # поисковый запрос
price_range = (1,100000) # ценовой диапазон

""" Создаем объект класса"""
driver = webdriver.Chrome(options=chrome_options)
# Открываем страницу с запросом
driver.get(f"https://www.avito.ru/{region}?p=1&q={search_query}")


# Получаем колличество страниц
items = driver.find_element(By.CLASS_NAME, 'items-items-kAJAg')
pages_count = items.find_element(By.XPATH, '//*[@id="app"]/div/div[4]/div/div[2]/div[3]/div[3]/div[4]').text.splitlines()[-1]
pages_count

for page_number in range(int(pages_count))[:3]:
    
    driver.get(f"https://www.avito.ru/{region}?p={page_number}&q={search_query}")

    # Получаем объект с картами объявлений
    items = driver.find_element(By.CLASS_NAME, 'items-items-kAJAg')

    # Получаем список объявлений
    elements = items.find_elements(By.CSS_SELECTOR,"[id^='i']")

    get_data(elements=elements)
    sleep(10)

1
Заголовок: Каркасный дом под ключ 68м2 Барнхаус68
Описание: Каркасный дом теплый контур за 1 Месяц строительства! ········. ·Договор, Допуски, Проект, Гарантия, Тех. Надзор··························. ·Все отзывы от клиентов на 5*·······················································. ·При заключении договора архитектурный проект в подарок·. ···································. Комплектация Стандарт: 1. Обрезная доска естественной влажности 150*50. 2. Утеплитель 100мм Rockwool «Стандарт». 3. Пароизоляция и ветрозащита Изоспан. 4. Кровля металочерепица. 5. Окна 2-х камерные, профиль 70мм. 6. Дверь металлическая Китай. 7. Винтовые сваи. Цена: 2 142 000. Комплектация Премиум: 1. Доска сухая строганная 200*50. 2. Утеплитель Rockwool «Скандик». 3. Пароизоляция сиж, скотч Delta. 4. Кровля Grand Line. 5. Окна Rehau 2-х камерные, провиль 70мм. 6. Дверь с терморазрывом. 7. Винтовые сваи. Цена: 3 060 000. Возможны Внесения Изменений В Любую Комплектацию. Проект Барн 68. Габариты (Ш*Д*В): 7м * 